# BIOL550 — Weekly Lab Report (STAR Alignment / Trapnell)

**Date:** 2026-02-10  \
**Student:** Piter Garcia (`pzg8794`)  \
**System:** `sequoia.rit.edu` (remote) + local workspace


## What Was Accomplished Since The Previous Report
- Completed QC (FastQC) on the Trapnell raw reads (12 FASTQ files) and packaged the HTML reports for submission.
- Used the reference genome distributed with the Trapnell zip and generated a STAR genome index from that FASTA+GTF after making chromosome naming consistent (BDGP6.84-style contig names).
- Completed STAR paired-end alignment for the Trapnell dataset (6 samples total: 3 C1 replicates + 3 C2 replicates).
- Verified each run produced `Aligned.sortedByCoord.out.bam` and `Log.final.out`, then packaged 1 representative BAM + `Log.final.out` for submission (per instructor guidance).


## Methods Used (Commands Used With Parameters)
The steps below document the exact commands and parameters used. Subheadings are organized by workflow stage.


### Step 0 — Connection + Paths (Server)

We run compute-heavy steps on `sequoia` and download only what we need for submission.

**Server:** `pzg8794@sequoia.rit.edu`


In [1]:
%%bash
set -euo pipefail

# Quick connectivity check (optional)
ssh pzg8794@sequoia.rit.edu 'hostname; whoami; pwd'


RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.


sequoia
pzg8794
/home/pzg8794


### Step 1 — Locate Inputs (Trapnell Data)

Raw reads directory (server):
- `/home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Raw reads/`

We expect 6 paired-end samples (C1_R1..R3 and C2_R1..R3), i.e. 12 gzipped FASTQs.


In [9]:
%%bash
set -euo pipefail

ssh pzg8794@sequoia.rit.edu <<'EOF'
set -euo pipefail
READS="/home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Raw reads"
ls -lh "$READS" | head
echo
echo "FASTQ count:" $(ls "$READS"/*.fq.gz 2>/dev/null | wc -l)
EOF

Pseudo-terminal will not be allocated because stdin is not a terminal.
RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.


Last login: Sun Feb  8 21:51:48 EST 2026 from 10.100.67.63 on ssh
Linux sequoia 6.12.63+deb13-amd64 #1 SMP PREEMPT_DYNAMIC Debian 6.12.63-1 (2025-12-30) x86_64

The programs included with the Debian GNU/Linux system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Debian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent
permitted by applicable law.
total 3.4G
-rw-rw-r-- 1 pzg8794 pzg8794 1.8G Sep 13  2011 GSM794483_C1_R1_1.fq
-rw-rw-r-- 1 pzg8794 pzg8794 138M Sep 13  2011 GSM794483_C1_R1_1.fq.gz
-rw-rw-r-- 1 pzg8794 pzg8794 138M Sep 13  2011 GSM794483_C1_R1_2.fq.gz
-rw-rw-r-- 1 pzg8794 pzg8794 138M Sep 13  2011 GSM794484_C1_R2_1.fq.gz
-rw-rw-r-- 1 pzg8794 pzg8794 138M Sep 13  2011 GSM794484_C1_R2_2.fq.gz
-rw-rw-r-- 1 pzg8794 pzg8794 138M Sep 13  2011 GSM794485_C1_R3_1.fq.gz
-rw-rw-r-- 1 pzg8794 pzg8794 138M Sep 13  2011 GSM794485_C1_R3_2.fq.gz
-rw-rw-r-- 1 pzg8794 pzg8794 139M Sep 13  2011 GS

### Step 1b — Read Length Check (One File)

We check one mate file to infer read length (used for `--sjdbOverhang = readLength - 1`).


In [5]:
%%bash
set -euo pipefail

ssh pzg8794@sequoia.rit.edu "READS='/home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Raw reads'; F=\"\$READS/GSM794486_C2_R1_1.fq.gz\"; zcat \"\$F\" | awk 'NR%4==2{print \"read_length\", length(\$0); exit}'"

RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.


read_length 75


### Step 2 — STAR Reference Prep (genomeGenerate)

STAR requires a genome index directory (`--genomeDir`).

We used the reference genome distributed with the Trapnell zip. Because the GTF uses BDGP6.84/FlyBase-style contig names (e.g., `2L/2R/3L/3R/...`) while the FASTA record IDs are RefSeq-style (`NC_/NT_/NW_...`), we first generated a renamed FASTA (`~/refs/classref_bdgp6_84_ids_v2.fa`) so the FASTA headers match the GTF seqnames.

Index directory used:
- `~/star_index/classref_trapnell_zip_bdgp6_84_v2`

If the index already exists, you do not need to re-run this step.


In [ ]:
%%bash
set -euo pipefail

ssh pzg8794@sequoia.rit.edu <<'EOF'
set -euo pipefail
REFDIR="/home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Drosophila reference"
FA="$REFDIR/GCF_000001215.fa"
GTF="$REFDIR/Drosophila_melanogaster.BDGP6.84.gtf"
RENAMED="$HOME/refs/classref_bdgp6_84_ids_v2.fa"
GENOMEDIR="$HOME/star_index/classref_trapnell_zip_bdgp6_84_v2"

echo "FA:" "$FA"
echo "GTF:" "$GTF"
echo "Renamed FASTA:" "$RENAMED"
echo "GenomeDir:" "$GENOMEDIR"

ls -lh "$FA" "$GTF" "$RENAMED" 2>/dev/null || true
ls -lh "$GENOMEDIR"/Genome "$GENOMEDIR"/SA "$GENOMEDIR"/SAindex 2>/dev/null || true
EOF


#### Step 2a — Implementation (genomeGenerate)

This is the command used (run with `nohup` on the server):


In [ ]:
%%bash
set -euo pipefail

ssh pzg8794@sequoia.rit.edu 'bash -s' <<'EOF'
set -euo pipefail
GENOMEDIR="$HOME/star_index/classref_trapnell_zip_bdgp6_84_v2"

if [ -f "$GENOMEDIR/SAindex" ]; then
  echo "STAR index already exists at $GENOMEDIR; skipping genomeGenerate."
else
  nohup /usr/local/bin/STAR/STAR --runMode genomeGenerate \
    --genomeDir "$GENOMEDIR" \
    --genomeFastaFiles "/home/pzg8794/refs/classref_bdgp6_84_ids_v2.fa" \
    --sjdbGTFfile "/home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Drosophila reference/Drosophila_melanogaster.BDGP6.84.gtf" \
    --sjdbOverhang 74 \
    --runThreadN 8 \
    --limitGenomeGenerateRAM 200000000000 \
    > "$GENOMEDIR/genomeGenerate.nohup.log" 2>&1 &
  echo "Started genomeGenerate; log: $GENOMEDIR/genomeGenerate.nohup.log"
fi
EOF

### Step 3 — STAR Alignment (One Sample)

We aligned one paired-end sample as a worked example:
- `GSM794486_C2_R1`

Key flags:
- `--readFilesCommand zcat` for `.gz` FASTQs
- `--outSAMstrandField intronMotif` (adds `XS` tags for spliced alignments; needed for `cuffdiff` downstream)
- `--outSAMtype BAM SortedByCoordinate`
- `--limitBAMsortRAM 6000000000` (required on this server to avoid BAM-sorting memory failure)


In [13]:
%%bash
set -euo pipefail

ssh pzg8794@sequoia.rit.edu 'bash -s' <<'EOF'
set -euo pipefail
READS="/home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Raw reads"
OUT="$HOME/BIOL550/Lab1/star_align_classref_v2_all_xs"
SAMPLE="GSM794486_C2_R1"

mkdir -p "$OUT/$SAMPLE"

echo "Reads dir: $READS"
echo "Output dir: $(readlink -f "$OUT/$SAMPLE")"

if [ -f "$OUT/$SAMPLE/Aligned.sortedByCoord.out.bam" ]; then
  echo "BAM already exists: $(readlink -f "$OUT/$SAMPLE/Aligned.sortedByCoord.out.bam")"
  echo "Skipping STAR."
  exit 0
fi

nohup /usr/local/bin/STAR/STAR \
  --genomeDir "$HOME/star_index/classref_trapnell_zip_bdgp6_84_v2" \
  --sjdbGTFfile "/home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Drosophila reference/Drosophila_melanogaster.BDGP6.84.gtf" \
  --runThreadN 4 \
  --readFilesIn "$READS/${SAMPLE}_1.fq.gz" "$READS/${SAMPLE}_2.fq.gz" \
  --readFilesCommand zcat \
  --outSAMstrandField intronMotif \
  --outSAMtype BAM SortedByCoordinate \
  --limitBAMsortRAM 6000000000 \
  --outFileNamePrefix "$OUT/$SAMPLE/" \
  > "$OUT/$SAMPLE/star.nohup.log" 2>&1 &

echo "Started STAR for $SAMPLE"
echo "Log: $(readlink -f "$OUT/$SAMPLE/star.nohup.log")"
EOF

RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.


Reads dir: /home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Raw reads
Output dir: /home/pzg8794/BIOL550/Lab1/star_align/GSM794486_C2_R1
BAM already exists: /home/pzg8794/BIOL550/Lab1/star_align/GSM794486_C2_R1/Aligned.sortedByCoord.out.bam
Skipping STAR.


#### Step 3a — Checks (Alignment Outputs)

After the run finishes, check:
- BAM exists and is non-empty
- `Log.final.out` exists (contains mapping percentages)


In [14]:
%%bash
set -euo pipefail

ssh pzg8794@sequoia.rit.edu bash -lc '
  set -euo pipefail
  OUT="$HOME/BIOL550/Lab1/star_align_classref_v2_all"
  SAMPLE="GSM794486_C2_R1"

  ls -lh "$OUT/$SAMPLE/Aligned.sortedByCoord.out.bam" || true
  ls -lh "$OUT/$SAMPLE/Log.final.out" || true
  echo
  if [ -f "$OUT/$SAMPLE/Log.final.out" ]; then
    cat "$OUT/$SAMPLE/Log.final.out" | sed -n "1,40p"
  fi
'

RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.


-rw-rw-r-- 1 pzg8794 pzg8794 409M Feb  5 10:47 /home/pzg8794/BIOL550/Lab1/star_align/GSM794486_C2_R1/Aligned.sortedByCoord.out.bam


bash: -c: option requires an argument


-rw-rw-r-- 1 pzg8794 pzg8794 1.9K Feb  5 10:47 /home/pzg8794/BIOL550/Lab1/star_align/GSM794486_C2_R1/Log.final.out

                                 Started job on |	Feb 05 10:44:31
                             Started mapping on |	Feb 05 10:44:32
                                    Finished on |	Feb 05 10:47:20
       Mapping speed, Million of reads per hour |	248.73

                          Number of input reads |	11607325
                      Average input read length |	150
                                    UNIQUE READS:
                   Uniquely mapped reads number |	11516696
                        Uniquely mapped reads % |	99.22%
                          Average mapped length |	149.99
                       Number of splices: Total |	3126338
            Number of splices: Annotated (sjdb) |	3110710
                       Number of splices: GT/AG |	3102242
                       Number of splices: GC/AG |	22129
                       Number of splices: AT/AC |	1346
       

### Step 3b — Loop Template (Align All Samples)

Use this after you confirm the single-sample run works.

This loops over all `*_1.fq.gz` mates and runs STAR on the paired `*_2.fq.gz`.


In [16]:
%%bash
set -euo pipefail

ssh pzg8794@sequoia.rit.edu 'bash -s' <<'EOF'
set -euo pipefail
READS="/home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Raw reads"
OUT="$HOME/BIOL550/Lab1/star_align_classref_v2_all_xs"
mkdir -p "$OUT"

for r1 in "$READS"/*_1.fq.gz; do
  base="$(basename "$r1" _1.fq.gz)"
  r2="$READS/${base}_2.fq.gz"
  [ -f "$r2" ] || { echo "Missing mate for $r1"; continue; }

  mkdir -p "$OUT/$base"

  if [ -f "$OUT/$base/Aligned.sortedByCoord.out.bam" ]; then
    echo "Exists, skipping: $OUT/$base/Aligned.sortedByCoord.out.bam"
    continue
  fi

  nohup /usr/local/bin/STAR/STAR \
    --genomeDir "$HOME/star_index/classref_trapnell_zip_bdgp6_84_v2" \
  --sjdbGTFfile "/home/pzg8794/BIOL550/Lab1/Trapnell_Data/Trapnell Data/Drosophila reference/Drosophila_melanogaster.BDGP6.84.gtf" \
    --runThreadN 4 \
    --readFilesIn "$r1" "$r2" \
    --readFilesCommand zcat \
    --outSAMstrandField intronMotif \
    --outSAMtype BAM SortedByCoordinate \
    --limitBAMsortRAM 6000000000 \
    --outFileNamePrefix "$OUT/$base/" \
    > "$OUT/$base/star.nohup.log" 2>&1 &

  echo "Started: $base"
done
EOF

RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.


Exists, skipping: /home/pzg8794/BIOL550/Lab1/star_align/GSM794483_C1_R1/Aligned.sortedByCoord.out.bam
Exists, skipping: /home/pzg8794/BIOL550/Lab1/star_align/GSM794484_C1_R2/Aligned.sortedByCoord.out.bam
Exists, skipping: /home/pzg8794/BIOL550/Lab1/star_align/GSM794485_C1_R3/Aligned.sortedByCoord.out.bam
Exists, skipping: /home/pzg8794/BIOL550/Lab1/star_align/GSM794486_C2_R1/Aligned.sortedByCoord.out.bam
Exists, skipping: /home/pzg8794/BIOL550/Lab1/star_align/GSM794487_C2_R2/Aligned.sortedByCoord.out.bam
Exists, skipping: /home/pzg8794/BIOL550/Lab1/star_align/GSM794488_C2_R3/Aligned.sortedByCoord.out.bam


### Step 3c — Package Alignment Files For Submission

Typical STAR deliverables per sample:
- `Aligned.sortedByCoord.out.bam`
- `Log.final.out`
- `Log.out`
- `SJ.out.tab`

This creates a single tarball on `sequoia` containing all available samples’ alignment outputs.


In [11]:
%%bash
set -euo pipefail

ssh pzg8794@sequoia.rit.edu 'bash -s' <<'EOF'
set -euo pipefail
EXPORT="$HOME/BIOL550/BIOL550-Lab/lab2"
OUT="$EXPORT/ALL_STAR_alignments.tgz"
mkdir -p "$EXPORT"

if [ -f "$OUT" ]; then
  echo "Archive exists, skipping: $OUT"
  ls -lh "$OUT"
  exit 0
fi

tar -czf "$OUT" \
  -C "$HOME/BIOL550/Lab1/star_align" \
  GSM794483_C1_R1/Aligned.sortedByCoord.out.bam GSM794483_C1_R1/Log.final.out \
  GSM794484_C1_R2/Aligned.sortedByCoord.out.bam GSM794484_C1_R2/Log.final.out \
  GSM794485_C1_R3/Aligned.sortedByCoord.out.bam GSM794485_C1_R3/Log.final.out \
  GSM794486_C2_R1/Aligned.sortedByCoord.out.bam GSM794486_C2_R1/Log.final.out \
  GSM794487_C2_R2/Aligned.sortedByCoord.out.bam GSM794487_C2_R2/Log.final.out \
  GSM794488_C2_R3/Aligned.sortedByCoord.out.bam GSM794488_C2_R3/Log.final.out

ls -lh "$OUT"
EOF

RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.


Archive exists, skipping: /home/pzg8794/BIOL550/BIOL550-Lab/lab2/ALL_STAR_alignments.tgz
-rw-rw-r-- 1 pzg8794 pzg8794 2.4G Feb  8 23:35 /home/pzg8794/BIOL550/BIOL550-Lab/lab2/ALL_STAR_alignments.tgz


In [6]:
%%bash
set -euo pipefail

LOCAL_EXPORT="/Users/pitergarcia/DataScience/Semester5/BIOL550/BIOL550-Lab/lab2/exports"
LOCAL_OUT="$LOCAL_EXPORT/star_alignment_files.tgz"
REMOTE_OUT="/home/pzg8794/BIOL550/Lab2/exports/star_alignment_files.tgz"

mkdir -p "$LOCAL_EXPORT"

# Check remote first
REMOTE_EXISTS=$(ssh pzg8794@sequoia.rit.edu 'test -f /home/pzg8794/BIOL550/Lab2/exports/star_alignment_files.tgz && echo yes || echo no')
if [ "$REMOTE_EXISTS" = "no" ]; then
  ssh pzg8794@sequoia.rit.edu 'bash -s' <<'EOF'
set -euo pipefail
EXPORT="/home/pzg8794/BIOL550/Lab2/exports"
OUT="$EXPORT/star_alignment_files.tgz"
mkdir -p "$EXPORT"
cd "/home/pzg8794/BIOL550/Lab1"

tar -czf "$OUT" \
  star_align/*/Aligned.sortedByCoord.out.bam \
  star_align/*/Log.final.out \
  star_align/*/Log.out \
  star_align/*/SJ.out.tab

ls -lh "$OUT"
EOF
else
  echo "Remote archive already exists: $REMOTE_OUT"
fi

# If local doesn’t exist, download it
if [ ! -f "$LOCAL_OUT" ]; then
  scp pzg8794@sequoia.rit.edu:"$REMOTE_OUT" "$LOCAL_OUT"
fi

ls -lh "$LOCAL_OUT"

RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.


Remote archive already exists: /home/pzg8794/BIOL550/Lab2/exports/star_alignment_files.tgz
-rw-r--r--@ 2 pitergarcia  staff   2.3G Feb  9 00:01 /Users/pitergarcia/DataScience/Semester5/BIOL550/BIOL550-Lab/lab2/exports/star_alignment_files.tgz


In [10]:
%%bash
set -euo pipefail

SAMPLE="GSM794483_C1_R1"
LOCAL_EXPORT="/Users/pitergarcia/DataScience/Semester5/BIOL550/BIOL550-Lab/lab2/exports"
LOCAL_OUT="$LOCAL_EXPORT/${SAMPLE}_STAR_alignment.tgz"
REMOTE_OUT="/home/pzg8794/BIOL550/Lab2/exports/${SAMPLE}_STAR_alignment.tgz"

mkdir -p "$LOCAL_EXPORT"

echo "Remote: $REMOTE_OUT"
echo "Local:  $LOCAL_OUT"

REMOTE_EXISTS=$(ssh pzg8794@sequoia.rit.edu "test -f '$REMOTE_OUT' && echo yes || echo no")
if [ "$REMOTE_EXISTS" = "no" ]; then
  ssh pzg8794@sequoia.rit.edu 'bash -s' <<EOF
set -euo pipefail
mkdir -p "/home/pzg8794/BIOL550/Lab2/exports"
tar -czf "$REMOTE_OUT" \
  -C "/home/pzg8794/BIOL550/Lab1/star_align/$SAMPLE" \
  Aligned.sortedByCoord.out.bam Log.final.out
EOF
fi

if [ ! -f "$LOCAL_OUT" ]; then
  scp pzg8794@sequoia.rit.edu:"$REMOTE_OUT" "$LOCAL_OUT"
fi

ls -lh "$LOCAL_OUT"

Remote: /home/pzg8794/BIOL550/Lab2/exports/GSM794483_C1_R1_STAR_alignment.tgz
Local:  /Users/pitergarcia/DataScience/Semester5/BIOL550/BIOL550-Lab/lab2/exports/GSM794483_C1_R1_STAR_alignment.tgz


RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.
RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and Network Use and RIT's 
Information Security Policy and Standards. Use of RIT information technology 
resources may be monitored and unauthorized use is strictly prohibited.
RIT information technology resources are for the use of the RIT community only. 
By using RIT information technology resources you acknowledge that you have read 
and comply with RIT's Code of Conduct for Computer and 

-rw-r--r--@ 2 pitergarcia  staff   400M Feb  9 00:09 /Users/pitergarcia/DataScience/Semester5/BIOL550/BIOL550-Lab/lab2/exports/GSM794483_C1_R1_STAR_alignment.tgz


## What Problems You Encountered
1. **Problem:** STAR annotation/reference incompatibility (GTF did not match the genome FASTA). **Cause:** The Trapnell zip GTF uses FlyBase chromosome names (e.g., `2L/2R/3L/3R/...`), while the Trapnell FASTA record IDs are RefSeq-style (`NC_/NT_/NW_...`), so contig names did not match and STAR could not load exon annotations. **Resolved:** Renamed FASTA headers to match the GTF (`classref_bdgp6_84_ids_v2.fa`), then re-built the STAR index and re-ran alignments.
1. **Problem:** FastQC `FAIL` modules persisted after cleanup. **Cause:** `fastq_quality_trimmer` / `fastq_quality_filter` thresholds did not materially change the quality/GC signals for this dataset. **Resolved:** Documented "attempted + verified no change" and proceeded to alignment.
1. **Problem:** BAM sorting required explicit memory allocation. **Cause:** STAR sorting can fail or stall without sufficient RAM configured for BAM sorting on the server. **Resolved:** Set `--limitBAMsortRAM 6000000000` for each alignment run.
1. **Problem:** `cuffdiff` downstream required `XS` tags on spliced alignments. **Cause:** STAR BAMs may omit `XS` unless configured. **Resolved:** Re-ran STAR for the final differential-expression-ready BAMs with `--outSAMstrandField intronMotif` and wrote them to `~/BIOL550/Lab1/star_align_classref_v2_all_xs/` (used for `cuffdiff`).
1. **Problem:** Deliverable size limits. **Cause:** STAR BAM outputs are large. **Resolved:** Submitted 1 representative BAM + `Log.final.out` as instructed; archived the full set separately.


## Goals For The Coming Week
- Alignment deliverable submitted (1 representative paired-end STAR BAM + `Log.final.out`, per instructor guidance).
- Work on Step 3: gene read counting and differential expression using Step 2 and later from Trapnell.
- Determine how to run CummeRbund in my current local environment (for the Lab 3 notebook/report).
- Catch up on the assigned readings.
